### Convex Optimization Exercise  4 ~ Code

In [1]:
import numpy as np
import scipy as sp
from scipy.sparse import diags
from numpy import linalg
from scipy.sparse.linalg import eigsh

def f_x(x,A,b):
    f = (1/2)*np.dot(np.dot(x.T,A),x) - np.dot(b.T,x)
    return f

### Initialization

In [2]:
dim = 10**4 # dimensions needed
main_diag = [9 for i in range(dim)] # 9...9 main diagonal
low_upper_diag = [-1 for i in range(dim)] # 1st up and 1st down diagonal
diagonals = [main_diag,low_upper_diag,low_upper_diag] #all diagonals in one list
A = diags(diagonals,[0,1,-1]) # sparse matrix with scipy using above list
b = np.asarray([8 if (i ==0 or i == dim-1) else 7 for i in range(dim)]) # b = [8,7,...,7,8]
print(b)
print(b.shape)
x0= np.asarray([0.5 for i in range(dim)]) # x0 = 1/2... ή 0.5
print(x0.shape)
A.todense()
A = A.toarray()

[8 7 7 ... 7 7 8]
(10000,)
(10000,)


In [3]:
## Solution for x = 1...1

x_solution = np.asarray([1 for i in range(dim)])
minimum = f_x(x_solution,A,b)
minimum

-35001.0

### Finding eivenvalues of A

#### 1st way, calculation via eigsh function from library scipy.sparse using matrix. It can return only 1 eigenvalue , largest or smallest, whatever we choose:  (lmax - lmin) / (lmax + lmin) = 0.22222...

#### 2nd way using calculations of numerical2.pdf page 15

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "myimage.png")

In [ ]:
A_sparse = diags(diagonals,[0,1,-1]) # sparse matrix με scipy

eigenvalues_max, _ = eigsh(A_sparse, k=1,which='LM')
eigenvalues_min, _ = eigsh(A_sparse, k=1,which='SM')

In [ ]:
eigenvalues_max[0]

In [ ]:
eigenvalues_min[0]

### Saving eigenvalues so that we dont execute above commands again

In [6]:
eigenvalues_max = 10.999999901457116
eigenvalues_min = 7.000000098621385

#((λmax + λmin) / (λmax - λmin))
myvalue = (eigenvalues_max - eigenvalues_min  ) / (eigenvalues_max+ eigenvalues_min )
myvalue

# |Χ0- Χ_bar|A
#np.sqrt(np.dot(np.dot((x0-xnew).T,A),x0-xnew))

0.22222221126768257

### Gradient Descent Algorithm for square function

Algorithm was executed for 20 iterations

In [7]:
r = b - np.dot(A,x0)
a = np.dot(r.T,r) / np.dot(np.dot(A,r),r.T)
xold = x0
xnew = xold + a*r
k=0
error = (myvalue**k)*np.sqrt(np.dot(np.dot((xold-xnew).T,A),xold-xnew))
while error > 10**-10:
    r = r - a*np.dot(A,r)
    a = np.dot(r.T,r) / np.dot(np.dot(A,r),r.T)
    xold = xnew
    xnew = xold + a*r
    k+=1
    error = (myvalue**k)*np.sqrt(np.dot(np.dot((x0-xnew).T,A),x0-xnew))
print(k)
print(xnew)

19
[1. 1. 1. ... 1. 1. 1.]


Here we see the F value , when inserting the solution from gradient descent method

In [8]:
print(error)
sol1 = f_x(xnew,A,b)
sol1
#print(abs(f_x(x_solution,A,b) - f_x(xnew,A,b)) < 1e-15)

5.1343687914900205e-11


-35001.00000000001

### Conjugated gradient method

This method converged in 19 iterations too. The benefit of this method is that we know that in certain number of steps we get a convergence(in at least n steps). Also this method is a matrix - free method. Finally we can see that we will have less error between real solution and the converged than in the first algorithm. That is because error in conjugated gradient method from theorem 15 says that:

       e_k = xk - x_bar , ||e_k|| <= (2r^k)/(1 + r^(2k))||e_0||_{A}
       
       where r = sqrt(K2(A)-1) / sqrt(K2(A)+1)


In [10]:
r = b - np.dot(A,x0)
p = r
a = np.dot(p.T,r) / np.dot(np.dot(A,p),p.T)
xold = x0
xnew = xold + a*p
k=0
error2 = (myvalue**k)*np.sqrt(np.dot(np.dot((xold-xnew).T,A),xold-xnew))
while error2 > 10**-10:
    r = r - a*np.dot(A,p)
    betas = np.dot(np.dot(A,p).T,r)/np.dot(np.dot(A,p).T, p)
    p = r - np.dot(betas,p)
    a = np.dot(p.T,r) / np.dot(np.dot(A,p),p.T)
    xold = xnew
    xnew = xold + a*p
    k+=1
    error2 = (myvalue**k)*np.sqrt(np.dot(np.dot((x0-xnew).T,A),x0-xnew))
print(k)
print(xnew)

19
[1. 1. 1. ... 1. 1. 1.]


Checking F value with new solution

In [12]:
print(error2)
sol2 = f_x(xnew,A,b)
print(sol2)
print(abs(f_x(x_solution,A,b) - f_x(xnew,A,b)) < 1e-323) # very small error..

5.1343687914900185e-11
-35001.0
True


#### Comparison of 2 solutions  - Conclusions
    We can see that the 2nd method converged to a better solution
    Both algorithms needed 19 iterations to converge.

In [13]:
#1st solution
print("Difference between 1st method and solution : ",abs(minimum - sol1))
#2nd solution
print("Difference between 2st method and solution : ",abs(minimum - sol2))

Difference between 1st method and solution :  7.275957614183426e-12
Difference between 2st method and solution :  0.0


In [14]:
abs(minimum - sol1) < abs(minimum - sol2)

False

#### Calculating k , with formula [(lmax - lmin) / (lmax + lmin)]^k * ||Xo-X_bar||_norm(A) <= tol, where tol = 10**-10

In [11]:
k = round(np.log((10**-10) / np.sqrt(np.dot(np.dot((x0-x_solution).T,A),x0-x_solution))) / np.log(myvalue))
k

19.0

So we need 19 iterations to converge to the best solution using tolerance = 10**10